In [ ]:
from rdkit import Chem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit.Chem.rdchem import ChiralType

import os.path as osp
import numpy as np
import glob
import pickle
import random

import networkx as nx
import torch_geometric as tg
from torch_geometric.utils import degree
from sklearn.utils import shuffle
import torch
import torch.nn.functional as F
from torch_scatter import scatter
from torch_geometric.data import Dataset, Data, DataLoader, InMemoryDataset

from typing import Any, Callable, Dict, Iterable, List, Optional, Tuple, Union
from tqdm import tqdm
from torch.nn.functional import one_hot

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
dihedral_pattern = Chem.MolFromSmarts('[*]~[*]~[*]~[*]')
chirality = {ChiralType.CHI_TETRAHEDRAL_CW: -1.,
             ChiralType.CHI_TETRAHEDRAL_CCW: 1.,
             ChiralType.CHI_UNSPECIFIED: 0,
             ChiralType.CHI_OTHER: 0}


def one_k_encoding(value, choices):
    """
    Creates a one-hot encoding with an extra category for uncommon values.
    :param value: The value for which the encoding should be one.
    :param choices: A list of possible values.
    :return: A one-hot encoding of the :code:`value` in a list of length :code:`len(choices) + 1`.
             If :code:`value` is not in :code:`choices`, then the final element in the encoding is 1.
    """
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1

    return encoding

class qm9_data(InMemoryDataset):
    """
    This is a class that extends the InMemoryDataset class from pytorch that will help later to 
    batch the data for training so this class will take the path where the data is an 
    stores each graph as a single data object.
    """
    def __init__(self, root: str, transform: Optional[Callable] = None,
                 pre_transform: Optional[Callable] = None,
                 pre_filter: Optional[Callable] = None):
        self.all_files = sorted(glob.glob(osp.join(root+'/qm9/', '*.pickle')))
        self.pickle_files = [f for i, f in enumerate(self.all_files)]
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        

    def process(self):
        data_list = []
        counter = 0
        for i in tqdm(self.pickle_files):
            data = featurization(i, counter)
            if self.pre_filter is not None and not self.pre_filter(data):
                continue
            if self.pre_transform is not None:
                data = self.pre_transform(data)
            if not(data == None):
                if isinstance(data, list):
                    for i in data:
                        data_list.append(i)
                        counter = counter + 1
                else:
                    data_list.append(data)
                    counter = counter + 1
            if counter == 500:
                break
            
        
        #data = self.collate(data_list)
        
        torch.save(self.collate(data_list), self.processed_paths[0])
        print("DataList length is: "+str(len(data_list)))
        
    
    
    def open_pickle(self, mol_path):
        # reads the mol pickle file and return the dictionary containing the data
        with open(mol_path, "rb") as f:
            dic = pickle.load(f)
        return dic
    
    @property
    def raw_file_names(self) -> List[str]:
        return self.all_files

    @property
    def processed_file_names(self):
        return ['data_v3.pt']
    def get_idx_split(self, data_size, train_size, valid_size, seed):
        ids = shuffle(range(data_size), random_state=seed)
        train_idx, val_idx, test_idx = torch.tensor(ids[:train_size]), torch.tensor(ids[train_size:train_size + valid_size]), torch.tensor(ids[train_size + valid_size:])
        split_dict = {'train':train_idx, 'valid':val_idx, 'test':test_idx}
        return split_dict
    
def featurization(mol_path: str, counter: int):
    chirality = {ChiralType.CHI_TETRAHEDRAL_CW: -1.,
             ChiralType.CHI_TETRAHEDRAL_CCW: 1.,
             ChiralType.CHI_UNSPECIFIED: 0,
             ChiralType.CHI_OTHER: 0}
    types = {'H': 0, 'C': 1, 'N': 2, 'O': 3, 'F': 4}

    dihedral_pattern = Chem.MolFromSmarts('[*]~[*]~[*]~[*]')
    types = {'H': 0, 'C': 1, 'N': 2, 'O': 3, 'F': 4}
    bonds = {BT.SINGLE: 0, BT.DOUBLE: 1, BT.TRIPLE: 2, BT.AROMATIC: 3}
    file = open(mol_path, 'rb')
    mol_dic = pickle.load(file)

    file.close()
    #save conformers of the moelcule
    confs = mol_dic['conformers']
    name = mol_dic["smiles"]
    confs = mol_dic['conformers']
    random.shuffle(confs)  # shuffle confs
    name = mol_dic["smiles"]
    max_confs=10
    # filter mols rdkit can't intrinsically handle
    mol_ = Chem.MolFromSmiles(name)
    if mol_:
        canonical_smi = Chem.MolToSmiles(mol_)
    else:
        return None

    # skip conformers with fragments
    if '.' in name:
        return None

    # skip conformers without dihedrals
    N = confs[0]['rd_mol'].GetNumAtoms()
    if N < 4:
        return None
    if confs[0]['rd_mol'].GetNumBonds() < 4:
        return None
    if not confs[0]['rd_mol'].HasSubstructMatch(dihedral_pattern):
        return None

    pos = torch.zeros([max_confs, N, 3])
    pos_mask = torch.zeros(max_confs, dtype=torch.int64)
    k = 0
    for conf in confs:
        mol = conf['rd_mol']

        # skip mols with atoms with more than 4 neighbors for now
        n_neighbors = [len(a.GetNeighbors()) for a in mol.GetAtoms()]
        if np.max(n_neighbors) > 4:
            continue

        # filter for conformers that may have reacted
        try:
            conf_canonical_smi = Chem.MolToSmiles(Chem.RemoveHs(mol))
        except Exception as e:
            continue

        if conf_canonical_smi != canonical_smi:
            continue

        pos[k] = torch.tensor(mol.GetConformer().GetPositions(), dtype=torch.float)
        pos_mask[k] = 1
        k += 1
        correct_mol = mol
        if k == 10:
            break

    # return None if no non-reactive conformers were found
    if k == 0:
        return None

    type_idx = []
    atomic_number = []
    atom_features = []
    chiral_tag = []
    neighbor_dict = {}
    ring = correct_mol.GetRingInfo()
    for i, atom in enumerate(correct_mol.GetAtoms()):
        type_idx.append(types[atom.GetSymbol()])
        n_ids = [n.GetIdx() for n in atom.GetNeighbors()]
        if len(n_ids) > 1:
            neighbor_dict[i] = torch.tensor(n_ids)
        chiral_tag.append(chirality[atom.GetChiralTag()])
        atomic_number.append(atom.GetAtomicNum())
        atom_features.extend([atom.GetAtomicNum(),
                                1 if atom.GetIsAromatic() else 0])
        atom_features.extend(one_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6]))
        atom_features.extend(one_k_encoding(atom.GetHybridization(), [
                                Chem.rdchem.HybridizationType.SP,
                                Chem.rdchem.HybridizationType.SP2,
                                Chem.rdchem.HybridizationType.SP3,
                                Chem.rdchem.HybridizationType.SP3D,
                                Chem.rdchem.HybridizationType.SP3D2]))
        atom_features.extend(one_k_encoding(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6]))
        atom_features.extend(one_k_encoding(atom.GetFormalCharge(), [-1, 0, 1]))
        atom_features.extend([int(ring.IsAtomInRingOfSize(i, 3)),
                                int(ring.IsAtomInRingOfSize(i, 4)),
                                int(ring.IsAtomInRingOfSize(i, 5)),
                                int(ring.IsAtomInRingOfSize(i, 6)),
                                int(ring.IsAtomInRingOfSize(i, 7)),
                                int(ring.IsAtomInRingOfSize(i, 8))])
        atom_features.extend(one_k_encoding(int(ring.NumAtomRings(i)), [0, 1, 2, 3]))

    z = torch.tensor(atomic_number, dtype=torch.long)
    chiral_tag = torch.tensor(chiral_tag, dtype=torch.float)

    row, col, edge_type, bond_features = [], [], [], []
    for bond in correct_mol.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        row += [start, end]
        col += [end, start]
        edge_type += 2 * [bonds[bond.GetBondType()]]
        bt = tuple(sorted([bond.GetBeginAtom().GetAtomicNum(), bond.GetEndAtom().GetAtomicNum()])), bond.GetBondTypeAsDouble()
        bond_features += 2 * [int(bond.IsInRing()),
                                int(bond.GetIsConjugated()),
                                int(bond.GetIsAromatic())]

    edge_index = torch.tensor([row, col], dtype=torch.long)
    edge_type = torch.tensor(edge_type, dtype=torch.long)
    edge_attr = one_hot(edge_type, num_classes=len(bonds)).to(torch.float)
    # bond_features = torch.tensor(bond_features, dtype=torch.float).view(len(bond_type), -1)

    perm = (edge_index[0] * N + edge_index[1]).argsort()
    edge_index = edge_index[:, perm]
    edge_type = edge_type[perm]
    # edge_attr = torch.cat([edge_attr[perm], bond_features], dim=-1)
    edge_attr = edge_attr[perm]

    row, col = edge_index
    hs = (z == 1).to(torch.float)
    #num_hs = scatter(hs[row], col, dim_size=N).tolist()

    x1 = one_hot(torch.tensor(type_idx), num_classes=len(types))
    x2 = torch.tensor(atom_features).view(N, -1)
    x = torch.cat([x1.to(torch.float), x2], dim=-1)
    y=conf['boltzmannweight']
    if len(pos)>1:
        data_list = []
        for i in pos:
            if not(torch.all(torch.eq(i, 0))):
                data = Data(x=x, z=z, pos=i, edge_index=edge_index, edge_attr=edge_attr,  
                    chiral_tag=chiral_tag, name=name, boltzmann_weight=conf['boltzmannweight'],
                    degeneracy=conf['degeneracy'], mol=correct_mol, pos_mask=pos_mask, y=y)
                data.edge_index_dihedral_pairs = get_dihedral_pairs(data.edge_index, data=data)
                data_list.append(data)
        return data_list
    else:
        if not(torch.all(torch.eq(pos[0], 0))):
            data = Data(x=x, z=z, pos=pos[0], edge_index=edge_index, edge_attr=edge_attr, 
                        chiral_tag=chiral_tag, name=name, boltzmann_weight=conf['boltzmannweight'],
                        degeneracy=conf['degeneracy'], mol=correct_mol, pos_mask=pos_mask,y=y)
        
        data.edge_index_dihedral_pairs = get_dihedral_pairs(data.edge_index, data=data)
        return data

def get_cycle_values(cycle_list, start_at=None):
    start_at = 0 if start_at is None else cycle_list.index(start_at)
    while True:
        yield cycle_list[start_at]
        start_at = (start_at + 1) % len(cycle_list)


def get_cycle_indices(cycle, start_idx):
    cycle_it = get_cycle_values(cycle, start_idx)
    indices = []

    end = 9e99
    start = next(cycle_it)
    a = start
    while start != end:
        b = next(cycle_it)
        indices.append(torch.tensor([a, b]))
        a = b
        end = b

    return indices
def get_current_cycle_indices(cycles, cycle_check, idx):
    c_idx = [i for i, c in enumerate(cycle_check) if c][0]
    current_cycle = cycles.pop(c_idx)
    current_idx = current_cycle[(np.array(current_cycle) == idx.item()).nonzero()[0][0]]
    return get_cycle_indices(current_cycle, current_idx)

def get_dihedral_pairs(edge_index, data):
    """
    Given edge indices, return pairs of indices that we must calculate dihedrals for
    """
    start, end = edge_index
    degrees = degree(end)
    dihedral_pairs_true = torch.nonzero(torch.logical_and(degrees[start] > 1, degrees[end] > 1))
    dihedral_pairs = edge_index[:, dihedral_pairs_true].squeeze(-1)
    
    # # first method which removes one (pseudo) random edge from a cycle
    dihedral_idxs = torch.nonzero(dihedral_pairs.sort(dim=0).indices[0, :] == 0).squeeze().detach().cpu().numpy()

    # prioritize rings for assigning dihedrals
    dihedral_pairs = dihedral_pairs.t().cpu().detach()[dihedral_idxs]
    G = nx.to_undirected(tg.utils.to_networkx(data))
    cycles = nx.cycle_basis(G)
    keep, sorted_keep = [], []

    if len(dihedral_pairs.shape) == 1:
        dihedral_pairs = dihedral_pairs.unsqueeze(0)

    for pair in dihedral_pairs:
        x, y = pair

        if sorted(pair) in sorted_keep:
            continue

        y_cycle_check = [y in cycle for cycle in cycles]
        x_cycle_check = [x in cycle for cycle in cycles]

        if any(x_cycle_check) and any(y_cycle_check):  # both in new cycle
            cycle_indices = get_current_cycle_indices(cycles, x_cycle_check, x)
            keep.extend(cycle_indices)

            sorted_keep.extend([sorted(c.cpu()) for c in cycle_indices])
            continue

        if any(y_cycle_check):
            cycle_indices = get_current_cycle_indices(cycles, y_cycle_check, y)
            keep.append(pair)
            keep.extend(cycle_indices)

            sorted_keep.append(sorted(pair))
            sorted_keep.extend([sorted(c.cpu()) for c in cycle_indices])
            continue

        keep.append(pair)

    #keep = torch.tensor(keep).to(device) 
    keep = [torch.tensor(t).to(device) for t in keep]
    
    return torch.stack(keep).t()

In [ ]:
import importlib
import torch
from torch_geometric.datasets import QM9
import pickle
from rdkit import Chem
import torch_geometric.data.data
from torch_geometric.data import Dataset, Data, Batch, InMemoryDataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
import os
import os.path as osp
import sys
from typing import Any, Callable, Dict, Iterable, List, Optional, Tuple, Union
from torch_geometric.data.collate import collate
from tqdm import tqdm
import glob
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit import Chem
from torch.nn.functional import one_hot
from torch import scatter
import random
from rdkit.Chem.rdchem import ChiralType
import numpy as np
from torchvision import transforms
from torch import Tensor
from dig.threedgraph.method import SphereNet
from dig.threedgraph import method
from dig.threedgraph.evaluation import ThreeDEvaluator
from dig.threedgraph.method import run
#from ../Project.data_process.qm9_dataset.py import qm9_data_geom
from qm9_dataset import qm9_data_geom

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
device

In [ ]:
path_to_data = '../../../others_approaches/conformation_generation/GeoMol/data/QM9/'


In [ ]:
qm9_set = qm9_data_geom(root= path_to_data)

In [ ]:
qm9_set[522125-3]

In [ ]:
target = 'boltzmann_weight' # mu, alpha, homo, lumo, gap, r2, zpve, U0, U, H, G, Cv
qm9_set.data.y = qm9_set.data[target]

split_idx = qm9_set.get_idx_split(len(qm9_set.data.y)-2, train_size=300, valid_size=100, seed=42)

train_dataset, valid_dataset, test_dataset = qm9_set[split_idx['train']], qm9_set[split_idx['valid']], qm9_set[split_idx['test']]


In [ ]:
model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4, 
        hidden_channels=128, out_channels=1, int_emb_size=64, 
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256, 
        num_spherical=3, num_radial=6, envelope_exponent=5, 
        num_before_skip=1, num_after_skip=2, num_output_layers=3, use_node_features=True
        )
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

In [ ]:
run3d = run()
run3d.run(device, train_dataset, valid_dataset, test_dataset, 
        model, loss_func, evaluation, 
        epochs=2, batch_size=4, vt_batch_size=4, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15)